In [2]:
import pandas as pd
import numpy as np
from tmdbv3api import Movie, TMDb
import json
import requests
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

In [48]:
df = pd.read_csv('d_2020.csv')
df.dropna(how = 'any')
df = df.drop_duplicates(keep = 'first', inplace = False)
df

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name
0,James Cameron,Joel David Moore,Action Adventure Fantasy Sci-Fi,CCH Pounder,avatar,Wes Studi
1,Gore Verbinski,Orlando Bloom,Action Adventure Fantasy,Johnny Depp,pirates of the caribbean: at world's end,Jack Davenport
2,Sam Mendes,Rory Kinnear,Action Adventure Thriller,Christoph Waltz,spectre,Stephanie Sigman
3,Christopher Nolan,Christian Bale,Action Thriller,Tom Hardy,the dark knight rises,Joseph Gordon-Levitt
4,Andrew Stanton,Samantha Morton,Action Adventure Sci-Fi,Daryl Sabara,john carter,Polly Walker
...,...,...,...,...,...,...
5951,Greta Gerwig,Emma Watson,Drama Romance,Saoirse Ronan,Little Women,Florence Pugh
5952,Sam Mendes,Dean-Charles Chapman,War Drama Action Thriller,George MacKay,1917,Mark Strong
5953,Destin Daniel Cretton,Jamie Foxx,Drama,Michael B. Jordan,Just Mercy,Brie Larson
5954,Chinonye Chukwu,Wendell Pierce,Drama,Alfre Woodard,Clemency,Aldis Hodge


In [4]:
tmdb = TMDb()
tmdb.api_key = 'deb477a3c3471f6e4e39aff4352f0806'
tmdb_movie = Movie()

In [8]:
def get_keyword(x):
    keyword = []
    result = tmdb_movie.search(x)
    m_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(m_id, tmdb.api_key))
    data_json = response.json()
    return data_json

In [59]:
def get_keywords(x):
    result = tmdb_movie.search(x)
    try:
        m_id = result[0].id
        link = "https://www.themoviedb.org/movie/{}".format(m_id)
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        html = urllib.request.urlopen(link,context=ctx).read()
        file = BeautifulSoup(html,'html.parser')
        tags = file('a')
        keywords = []
        for tag in tags:
            line = tag.get('href', None)
            if '/keyword/' in line:
                keywords.append(line.split('/keyword/')[1].split('-')[0])
        return ' '.join(keywords)
    except:
        return np.NaN

In [60]:
df['keywords'] = df['movie_title'].map(lambda x: get_keywords(x))

In [64]:
df

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,keywords
0,James Cameron,Joel David Moore,Action Adventure Fantasy Sci-Fi,CCH Pounder,avatar,Wes Studi,1463 2964 3386 3388 3679 3801 9685 9840 9882 9...
1,Gore Verbinski,Orlando Bloom,Action Adventure Fantasy,Johnny Depp,pirates of the caribbean: at world's end,Jack Davenport,911 1319 2038 2052 2580 2660 4224 6155 12988 1...
2,Sam Mendes,Rory Kinnear,Action Adventure Thriller,Christoph Waltz,spectre,Stephanie Sigman,470 818 4289 9663 14555 156095 158431
3,Christopher Nolan,Christian Bale,Action Thriller,Tom Hardy,the dark knight rises,Joseph Gordon-Levitt,849 853 949 1308 1437 1562 1721 3562 7002 9665...
4,Andrew Stanton,Samantha Morton,Action Adventure Sci-Fi,Daryl Sabara,john carter,Polly Walker,818 839 1456 3801 7376 9951 10028 10539 10685 ...
...,...,...,...,...,...,...,...
5951,Greta Gerwig,Emma Watson,Drama Romance,Saoirse Ronan,Little Women,Florence Pugh,380 818 1700 5923 9714 13028 207928 241738
5952,Sam Mendes,Dean-Charles Chapman,War Drama Action Thriller,George MacKay,1917,Mark Strong,2504 4100 4776 13065 188725 210184 230915 2361...
5953,Destin Daniel Cretton,Jamie Foxx,Drama,Michael B. Jordan,Just Mercy,Brie Larson,934 6139 9672 10909 12425 15108 208289
5954,Chinonye Chukwu,Wendell Pierce,Drama,Alfre Woodard,Clemency,Aldis Hodge,378 2501 5643 187056


In [65]:
df.to_csv('final.csv', index = False)

In [8]:
df = pd.read_csv('final.csv')
df.replace(' ', 'unknown')
df.to_csv('final2.csv', index = False)